Install the required packages:

In [1]:
!pip install datasets
!pip install --upgrade gensim
!pip install langdetect
!pip install plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=b9fe55076e276392c2c53df15e91c4c9694699d0027f7d0d61c2cdf4f727120e
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


Import the required modules:

In [2]:
from datasets import load_dataset
from gensim.models.word2vec import Word2Vec
from langdetect import detect
from pandas.core.common import flatten
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
import plotly.express as px
import random
import re

Load the guanaco dataset:

In [3]:
dataset = load_dataset("timdettmers/openassistant-guanaco")

Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/518 [00:00<?, ? examples/s]

Let's gather the array of strings containing all the interactions in the dataset:

In [4]:
docs = dataset['train']['text'] + dataset['test']['text']

Let's define a function that returns a list of only the english strings:

In [5]:
def filter_english(strings):
  english_strings = []
  for string in strings:
    try:
      if detect(string) == 'en':
        english_strings.append(string)
    except:
      print(f"Error with {string}")
  return english_strings

We keep only the english interactions:

In [6]:
#This line will also output the only string that the function is unable to correctly identify
docs = filter_english(docs)

Error with ### Human: Generate me an ASCII of a person with a jetpack### Assistant: Here is an ASCII art representation of a person with a jetpack:

─────────▄███████████▄▄──────────────
──────▄██▀──────────▀▀██▄────────────
────▄█▀────────────────▀██───────────
──▄█▀────────────────────▀█▄─────────
─█▀──██──────────────██───▀██────────
█▀──────────────────────────██───────
█──███████████████████───────█───────
█────────────────────────────█───────
█────────────────────────────█───────
█────────────────────────────█───────
█────────────────────────────█───────
█────────────────────────────█───────
█▄───────────────────────────█───────
▀█▄─────────────────────────██───────
─▀█▄───────────────────────██────────
──▀█▄────────────────────▄█▀─────────
───▀█▄──────────────────██───────────
─────▀█▄──────────────▄█▀────────────
───────▀█▄▄▄──────▄▄▄███████▄▄───────
────────███████████████───▀██████▄───
─────▄███▀▀────────▀███▄──────█─███──
───▄███▄─────▄▄▄▄────███────▄▄████▀──
─▄███▓▓█─────█▓

An example of the output:

In [7]:
print(f"Only english interactions example:\n {docs[:5]}")

Only english interactions example:
 ['### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: "Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefit

Let's separate the sentences written by the human from the ones of the assistant:

In [8]:
human_questions = []
assistant_answers = []

for string in docs:
  segments = string.split("### Human:")
  for segment in segments[1:]:
    # This if satement has been added to append to the list of human questions only those that have then received an answer by the assistant
    if "### Assistant:" in segment:
      human_questions.append(segment.split("### Assistant:")[0].strip())

  segments = string.split("### Assistant:")
  for segment in segments[1:]:
      assistant_answers.append(segment.split("### Human:")[0].strip())

How many questions/anwers do we have?

In [9]:
print(len(human_questions))
print(len(assistant_answers))

5485
5485


Let's now clean the datasets and split each interaction into different sentences:

In [10]:
human_questions = [re.sub('[\n\t]', '', doc) for doc in human_questions]
assistant_answers = [re.sub('[\n\t]', '', doc) for doc in assistant_answers]
human_questions = [re.sub('[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', '', doc) for doc in human_questions]
assistant_answers = [re.sub('[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', '', doc) for doc in assistant_answers]
human_questions = [re.split('[?!.]\s', doc) for doc in human_questions]
assistant_answers = [re.split('[?!.]\s', doc) for doc in assistant_answers]

Flatten the list of lists and tokenize each word:

In [11]:
human_questions = list(flatten(human_questions))
human_questions = [re.sub('\W', ' ', doc).lower().split() for doc in human_questions]

assistant_answers = list(flatten(assistant_answers))
assistant_answers = [re.sub('\W', ' ', doc).lower().split() for doc in assistant_answers]

# We also compute a combination of the two tet
full_text = human_questions + assistant_answers

 The minimum length of each sentence is 5, so we don't need to remove sentences that are composed by few words

 Now we train the Word2Vec on the human questions, by providing as input the tokenized words, the size of each embedding, the minimum number of occurences for each word and the context window size:



In [12]:
model_human = Word2Vec(human_questions, vector_size=30, min_count=5, window=10)

How many words do we have in our model?

In [13]:
len(model_human.wv)

2919

How is each embedding vector made?

In [14]:
term = 'house'
model_human.wv[term]

array([-0.01373635, -0.46641207,  0.6761006 , -0.12135923,  0.56212527,
        0.1507072 ,  0.47833908,  0.24573052, -0.5097045 ,  0.07439438,
        0.3257118 , -0.09254123, -0.18814473, -0.5715499 , -0.08434059,
       -0.05991653,  0.48502994,  0.22369754, -0.606034  ,  0.18112504,
       -0.09862562, -0.34129328,  0.5079902 ,  0.7984394 ,  0.18167818,
        0.09822354,  0.27595955,  0.54621637,  0.23876965, -1.0097482 ],
      dtype=float32)

What are the most similar words to the word "short" ?

In [15]:
term = 'short'
model_human.wv.most_similar(term)

[('an', 0.9967186450958252),
 ('script', 0.9956920146942139),
 ('outline', 0.994983971118927),
 ('simple', 0.9943808913230896),
 ('summary', 0.9932616949081421),
 ('story', 0.9926920533180237),
 ('example', 0.9920281171798706),
 ('detailed', 0.9911024570465088),
 ('latex', 0.9908677339553833),
 ('poem', 0.9903829097747803)]

Now, let's do the same thing but with the assistant answers:

In [16]:
model_assistant = Word2Vec(assistant_answers, vector_size=30, min_count=5, window=10)

It's vocabulary size is more than 4 times the human one:

In [17]:
len(model_assistant.wv)

12094

Let's see an example of embedding:

In [18]:
term = 'house'
model_assistant.wv[term]

array([-0.32050413, -0.16883625,  0.6354963 ,  0.1762308 , -0.16047207,
       -0.20180887,  0.5534151 ,  0.1170984 , -0.73068166,  0.06779248,
        0.35919654,  0.18999384, -0.28415245, -0.20341255,  0.20017247,
       -0.11923487,  0.8492205 ,  0.44023806, -1.1120838 ,  0.7906691 ,
        0.02754617, -0.6524072 ,  0.48019323,  0.69135356,  0.39069825,
        0.79453254, -0.4708122 ,  0.9464931 ,  0.51499313, -0.70055217],
      dtype=float32)

And the most similar words to the word "short":

In [19]:
term = 'short'
model_assistant.wv.most_similar(term)

[('great', 0.8467867374420166),
 ('bonds', 0.8466100096702576),
 ('bird', 0.8441928029060364),
 ('octopus', 0.8438166975975037),
 ('juggling', 0.8428542613983154),
 ('ride', 0.8428131341934204),
 ('shooting', 0.8349483013153076),
 ('elusive', 0.8341455459594727),
 ('rare', 0.8326117396354675),
 ('exciting', 0.8322793841362)]

Finally, let's perform the same operations with the full_text to our disposal:

In [20]:
model_full = Word2Vec(full_text, vector_size=30, min_count=5, window=10)

The length of the vocabulary isn't much different from the one of just the assistant:

In [21]:
len(model_full.wv)

12930

Let's see again an example of an embedding:

In [22]:
term = 'plant'
model_full.wv[term]

array([-0.6082743 , -0.10587323,  1.3211502 ,  0.08273781, -0.17049456,
        0.05797599, -0.14557837, -0.46745518, -0.9932261 , -0.72588843,
        0.5137545 , -0.4750788 ,  0.44161746, -0.5824621 , -0.48752537,
       -0.2211853 ,  0.15561366, -0.16292208, -0.85695314,  0.27147532,
        0.18580976,  0.07909161, -0.22281215,  0.815345  ,  0.642691  ,
       -0.26438442,  0.44272608,  0.35202533,  0.09243715, -1.2175586 ],
      dtype=float32)

And once again the most similar word to the word short:

In [23]:
term = 'short'
model_full.wv.most_similar(term)

[('great', 0.8254386186599731),
 ('story', 0.7947911620140076),
 ('fun', 0.7811746597290039),
 ('song', 0.7746869921684265),
 ('joke', 0.7733042240142822),
 ('exciting', 0.7625746130943298),
 ('couple', 0.7530159950256348),
 ('humor', 0.7408021092414856),
 ('mini', 0.7363277077674866),
 ('baker', 0.7345274090766907)]

Has the model understood the relation between this words?

In [24]:
vec = model_full.wv['king'] + (model_full.wv['woman'] - model_full.wv['man'])
vec

array([-0.22763573,  0.07126992,  0.7812185 , -0.05194846, -0.5692307 ,
        0.17775209,  0.8479804 , -1.0273545 , -0.46933883, -0.08976364,
        1.0456717 ,  0.00498044, -0.10044368, -0.71537673, -0.2224139 ,
       -0.89263093,  1.0726649 ,  0.59544504, -0.11588743,  0.15679799,
       -0.19628918, -0.01117885,  0.40998054,  0.73787606, -0.30041355,
        0.36658666,  0.20838422,  0.5668458 , -0.7696963 , -0.9152121 ],
      dtype=float32)

Seems it does not:

In [25]:
model_full.wv.most_similar(vec)

[('toward', 0.9320296049118042),
 ('soldiers', 0.9179599285125732),
 ('spotted', 0.9144290685653687),
 ('daring', 0.9110090732574463),
 ('brave', 0.9077989459037781),
 ('penelope', 0.9035419821739197),
 ('glory', 0.9029262661933899),
 ('argues', 0.9019280672073364),
 ('seize', 0.8994498252868652),
 ('possesses', 0.8990603685379028)]

And this kind of relation?

In [26]:
vec = model_full.wv['france'] + (model_full.wv['rome'] - model_full.wv['italy'])
vec

array([-0.22523104,  0.15744157,  0.824334  , -0.10032804, -1.266954  ,
       -0.10496541,  1.362934  , -1.0151162 , -1.1713355 , -0.8996093 ,
        0.70536876,  0.36884868,  0.08915871, -0.29758146, -1.0982912 ,
       -1.1462512 ,  0.12532166,  0.23793629, -0.5558492 ,  0.24965599,
       -0.41790628,  0.91470736,  0.79918736,  1.2844195 , -0.33046436,
       -0.0519993 , -0.2256724 ,  0.65818995, -0.9430068 , -1.4931618 ],
      dtype=float32)

It seems to have a better idea of what we are talking about, but still he hasn't fully understood the relation:

In [27]:
model_full.wv.most_similar(vec)

[('france', 0.9766879677772522),
 ('rome', 0.9707320332527161),
 ('leader', 0.943225085735321),
 ('hitler', 0.9392644166946411),
 ('south', 0.9364587068557739),
 ('death', 0.9360859990119934),
 ('continent', 0.9356143474578857),
 ('independence', 0.9345585107803345),
 ('roman', 0.9331175684928894),
 ('tunisia', 0.9303480386734009)]

#Visualizing the embedding vector using t-SNE

We'll now produce a graphical representation of a subset of the embeddings, because to reduce the time required for computation, we'll limit our representation to 500 random samples:

In [28]:
random_samples_human = random.sample(list(model_human.wv.key_to_index), 500)
word_vectors_human = model_human.wv[random_samples_human]

random_samples_assistant = random.sample(list(model_assistant.wv.key_to_index), 500)
word_vectors_assistant = model_assistant.wv[random_samples_assistant]

random_samples_full = random.sample(list(model_full.wv.key_to_index), 500)
word_vectors_full = model_full.wv[random_samples_full]

We'll provide the vectors to the TSNE algorithm, to fit a model and have a 3 dimensional representation:

In [29]:
tsne = TSNE(n_components=3, n_iter=2000)

tsne_embeddings_human = tsne.fit_transform(word_vectors_human)
tsne_embeddings_assistant = tsne.fit_transform(word_vectors_assistant)
tsne_embeddings_full = tsne.fit_transform(word_vectors_full)

We transpose the matrix, as to have each dimension in each row of the resulting matrix and we get the coordinates of each point:

In [30]:
x_human, y_human, z_human = np.transpose(tsne_embeddings_human)
x_assistant, y_assistant, z_assistant = np.transpose(tsne_embeddings_assistant)
x_full, y_full, z_full = np.transpose(tsne_embeddings_full)

We draw the plots (we also reduced the amount of samples to be shown for a bettere representation):

In [31]:
fig = px.scatter_3d(x=x_human[:150], y=y_human[:150], z=z_human[:150], text=random_samples_human[:150])
fig.update_traces(marker=dict(size=3,line=dict(width=2)))
fig.show()

In [32]:
fig = px.scatter_3d(x=x_assistant[:150], y=y_assistant[:150], z=z_assistant[:150], text=random_samples_assistant[:150])
fig.update_traces(marker=dict(size=3,line=dict(width=2)))
fig.show()

In [33]:
fig = px.scatter_3d(x=x_full[:150], y=y_full[:150], z=z_full[:150], text=random_samples_full[:150])
fig.update_traces(marker=dict(size=3,line=dict(width=2)))
fig.show()